### 과적합
- 음파 탐지기 데이터 정보를 이용해 광석과 돌 구분하기 (p.159)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy
import tensorflow as tf

# seed 값 설정
numpy.random.seed(3)
tf.random.set_seed(3)

# 데이터 입력
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/sonar.csv', header=None)

# 데이터 개괄 보기
print(df.info())

# 총 208개의 샘플, 60개의 속성, 1개의 클래스로 이루어진 것을 알 수 있다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [10]:
# 데이터의 일부분 미리 보기
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [11]:
# 데이터의 일부분 미리 보기
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M
207,0.0260,0.0363,0.0136,0.0272,0.0214,0.0338,0.0655,0.1400,0.1843,0.2354,...,0.0146,0.0129,0.0047,0.0039,0.0061,0.0040,0.0036,0.0061,0.0115,M


In [16]:
# 입력데이터 처리
dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

# 문자열 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
#Y_encoded = tf.keras.utils.to_categorical(Y)
#print(Y_encoded)

# 모델 설정
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
#model.add(Dense(2, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['accuracy'])

# 모델 실행
model.fit(X, Y, epochs=200, batch_size=5)

# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

# 정확도가 100%가 나오는 것을 확인할 수 있다.
# ⇒ 과적합 : 학습 데이터셋 안에서는 높은 정확도를 보이지만, 새로운 데이터에 적용하면 잘 맞지 않는 것 
# 층이 너무 많거나, 변수가 복잡하거나, 테스트 셋과 학습 셋이 중복되는 등의 상황에 발생

# → 70% 샘플은 학습셋으로, 30%샘플은 테스트셋으로.
# 70%를 학습한 것이 저장된 모델을 30%샘플로 테스트해보며 정확도 확인.

Epoch 1/200
42/42 [==============================] - 0s 2ms/step - loss: 0.2559 - accuracy: 0.5240
Epoch 2/200
42/42 [==============================] - 0s 2ms/step - loss: 0.2447 - accuracy: 0.5625
Epoch 3/200
42/42 [==============================] - 0s 2ms/step - loss: 0.2364 - accuracy: 0.6154
Epoch 4/200
42/42 [==============================] - 0s 2ms/step - loss: 0.2237 - accuracy: 0.6827
Epoch 5/200
42/42 [==============================] - 0s 2ms/step - loss: 0.2111 - accuracy: 0.7548
Epoch 6/200
42/42 [==============================] - 0s 2ms/step - loss: 0.2014 - accuracy: 0.7452
Epoch 7/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1889 - accuracy: 0.7740
Epoch 8/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1778 - accuracy: 0.7596
Epoch 9/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1742 - accuracy: 0.7596
Epoch 10/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1654 - accuracy: 0.7933
Epoch 11/